# Charter_flights - Modelling

## Autores:
- Alejandro De Haro
- Daniel Escobosa
- Pablo Berastegui
- Jose María Benitez


### Importing packages and modules


In [1]:
import pyomo.environ as pe
import pyomo.opt as po

### Create the model

### Order to build the model:
1. Sets  
2. Parameters  
3. Variables  
4. Objective function  
5. Constraints  
6. Solver definition and solve statement  
7. Results

In [2]:
model = pe.ConcreteModel()

### Sets

- $n \in \{1,2,3,4,5\}$: **option** for the number of airplanes of a given type to charter.  
- $t \in \{\mathrm{A},\mathrm{B},\mathrm{C}\}$: **aircraft type**.

In [3]:
model.n = pe.Set(initialize=[1,2,3,4,5])
model.t = pe.Set(initialize=['A','B','C'])

### Parameters

- $c_{n,t}$ *(k€)*: **cost** associated with choosing option $n$ for aircraft type $t$.  
- $\mathrm{cap}_t$ *(pax/plane)*: **seat capacity** per airplane of type $t$.  
- $P$ *(pax)*: **total passenger demand** to be satisfied.  
- $FC$ *(k€)*: **fixed cost per selected option** (common add-on applied when an option is chosen).

In [4]:
cost_table = {
    (1,'A'):11, (2,'A'):20, (3,'A'):30, (4,'A'):40, (5,'A'):50,
    (1,'B'): 9, (2,'B'):17, (3,'B'):24, (4,'B'):34, (5,'B'):45,
    (1,'C'): 8, (2,'C'):15, (3,'C'):21, (4,'C'):26, (5,'C'):31
}
model.c = pe.Param(model.n, model.t, initialize=cost_table)

capacity = {'A':80, 'B':68, 'C':55}
model.cap = pe.Param(model.t, initialize=capacity)

model.P = pe.Param(initialize=372)
model.FC = pe.Param(initialize=6)  # k€

### Variables

- $x_{n,t} \in \{0,1\}$: equals 1 if **option $n$** is selected for **type $t$**, and 0 otherwise.


In [5]:
model.x = pe.Var(model.n, model.t, within=pe.Binary)

### Objective Function

We **minimize total cost** of the chosen options:

In [6]:
def obj_rule(model):
    return sum((model.c[n,t] + model.FC) * model.x[n,t] for n in model.n for t in model.t)
model.total_cost = pe.Objective(rule=obj_rule, sense=pe.minimize)

### Constraints

**(1) At most one option per aircraft type**

**(2) Demand satisfaction via total capacity**

In [7]:
def one_choice_per_type_rule(model, t):
    return sum(model.x[n,t] for n in model.n) <= 1
model.one_choice_per_type = pe.Constraint(model.t, rule=one_choice_per_type_rule)

def capacity_rule(model):
    return sum(n * model.cap[t] * model.x[n,t] for n in model.n for t in model.t) >= model.P
model.capacity_ok = pe.Constraint(rule=capacity_rule)

### Solver definition and solve statement

In [8]:
solver = po.SolverFactory('gurobi')
results = solver.solve(model, tee=True)

Set parameter Username
Set parameter LicenseID to value 2704768
Academic license - for non-commercial use only - expires 2026-09-08
Read LP format model from file /var/folders/pr/nst7vndn0l10vqp0s73wclhr0000gn/T/tmp3rsoyf8t.pyomo.lp
Reading time = 0.00 seconds
x1: 4 rows, 15 columns, 30 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 15 columns and 30 nonzeros
Model fingerprint: 0x9e6b927a
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective 60.0000000
Presolve time: 0.00s
Presolved: 4 rows, 15 columns, 30 nonzeros
Variable types: 0 continuous, 15 integer (15 binary)

Root relaxation: objective 5.296215e+01, 1 iterations, 0

### Results

We report:
- **Optimal total cost** (`model.total_cost`), in k€.
- **Selected number of airplanes per type** (derived from `x[n,t]`).
- **Total capacity** vs **demand** to verify feasibility.

The printed summary shows the chosen plan by aircraft type and the aggregate capacity check.

In [9]:
opt_cost = pe.value(model.total_cost)
print(f"Optimal total cost: {opt_cost:.2f} k€")

chosen = {}
for t in model.t:
    n_used = sum(n * pe.value(model.x[n,t]) for n in model.n)
    chosen[t] = int(round(n_used))
    print(f"Type {t}: {chosen[t]} airplanes selected")

total_capacity = sum(chosen[t] * pe.value(model.cap[t]) for t in model.t)
print(f"Total capacity: {int(total_capacity)} passengers (demand = {pe.value(model.P)})")

Optimal total cost: 56.00 k€
Type A: 5 airplanes selected
Type B: 0 airplanes selected
Type C: 0 airplanes selected
Total capacity: 400 passengers (demand = 372)
